In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas_datareader.data as web

In [2]:
def import_order_book(Path):
    columns = []
    for i in range(10):
        columns.append(("Ask Price "+ str(i)))
        columns.append(("Ask Size " + str(i)))
        columns.append(("Bid Price "+ str(i)))
        columns.append(("Bid Size "+ str(i)))
        
    Orderbook = pd.read_csv(Path, names = columns)
    
    Ask_Price = []
    Bid_Price = []
    for i in (Orderbook.columns):
        if i[:9] == "Ask Price":
            Ask_Price.append(i)
        if i[:9] == "Bid Price":
            Bid_Price.append(i)
    
    Bid_price = []
    for i in range(len(Bid_Price)-1,-1,-1):
        Bid_price.append(Bid_Price[i])
    Bid_Price = Bid_price
            
    Ask_size = []
    Bid_size = []
    for i in (Orderbook.columns):
        if i[:8] == "Ask Size":
            Ask_size.append(i)
        if i[:8] == "Bid Size":
            Bid_size.append(i)
    
    Bid_Size = []
    for i in range(len(Bid_size)-1,-1,-1):
        Bid_Size.append(Bid_size[i])
    Bid_size = Bid_Size
            
    Orderbook.loc[:,(Bid_size)] = -Orderbook.loc[:,(Bid_size)]
    
    Orderbook_price = Orderbook.loc[:, (Bid_Price + Ask_Price)]
    Orderbook_size = Orderbook.loc[:, (Bid_size + Ask_size)]
    return Orderbook_price, Orderbook_size


In [3]:
columns = ["time", "type", "order_id", "size", "price", "direction"]

#The messages in order book
df = pd.read_csv("AMZN_2012-06-21_34200000_57600000_message_10.csv", header = None, names = columns)

In [4]:
#We ytake away the message we don't want to treat
df_new = df[df.type != 5]

In [5]:
df_new.head()

,time,type,order_id,size,price,direction
1,34200.189608,1,11885113,21,2238100,1
2,34200.189608,1,3911376,20,2239600,-1
3,34200.189608,1,11534792,100,2237500,1
4,34200.189608,1,1365373,13,2240000,-1
5,34200.189608,1,11474176,2,2236500,1


## Definition of the problem

In [6]:
Directions = {-1: "Sell limit order",
              1: "Buy limit order"}
size = 20000
direction = 1
Time = 4000
actions = {1: "Submission of a new limit order",
           4: "Execution of a visible limit order"}
Actions = [1, 4]


## Load data

In [323]:
columns = ["time", "type", "order_id", "size", "price", "direction"]
obp, obs = import_order_book("AMZN_2012-06-21_34200000_57600000_orderbook_10.csv")
df = pd.read_csv("AMZN_2012-06-21_34200000_57600000_message_10.csv", header = None, names = columns)

## Training the model
This is the first training, if you want to train additional times, go and run the next part (Start here)

In [324]:
C = {}

In [356]:
A = [i*300 for i in range (-10,11)]
I = 4
V = 10000
T = 8
TT = 4000
N = 10 # Number of training iterations


for n in range (0,10):
    if n > 0 :
        obp = obp.iloc[TT+1:,:].reset_index(drop = True)
        obs = obs.iloc[TT+1:,:].reset_index(drop = True)
        df = df.iloc[TT+1:,:].reset_index(drop = True)
    m0 = int((obp.iloc[0, 9] +  obp.iloc[0, 10])/2) 
    for t in range(T,-1,-1):
        print("N : ", n, "time : ", t)
        Ls = obs.iloc[t*int(TT/T):(t+1)*int(TT/T),:]
        Lp = obp.iloc[t*int(TT/T):(t+1)*int(TT/T), :]
        O = df.iloc[t*int(TT/T):(t+1)*int(TT/T), :]

        for i in range(0, I+1):
            for a in A:
                s = (t, i)
                v_remaining = i*V/I
                if t == T: 
                    x = (v_remaining, 1)
                    C[(s,a)] = Lp.iloc[int(TT/T)-1, 10] * v_remaining - v_remaining * m0 
                else:
                    x = (v_remaining, a)
                    vt = []
                    mt = []

                    if Lp.iloc[0, 9] + a < Lp.iloc[0,10]:
                        for tt in range(t*int(TT/T), (t+1)*int(TT/T)):
                            if v_remaining > 0:
                                if ((O.loc[tt, "type"] == 4) & (O.loc[tt, "price"] <= Lp.iloc[0, 9] + a) & (O.loc[tt, "direction"] == 1)):
                                    vt.append(O.loc[tt, "size"])
                                    v_remaining -= O.loc[tt, "size"]
                                    mt.append(Lp.iloc[0, 9] + a)
                                if ((O.loc[tt, "type"] == 1) & (O.loc[tt, "price"] <= Lp.iloc[0, 9] + a) & (O.loc[tt, "direction"] == -1)):
                                    vt.append(O.loc[tt, "size"])
                                    v_remaining -= O.loc[tt, "size"]
                                    mt.append(O.loc[tt, "price"])
                        v_remaining = max (0, v_remaining)
                        ip = int(v_remaining/(V/I))

                        sp = (t+1, ip)
                        temp = [C[(sp, j)] for j in A]
                        for k in A:
                            if C[(sp, k)] == min(temp):
                                ap = k
                        if n == 0:
                            C[(s,a)] = (np.matmul(vt, mt))- m0 * sum(vt) + C[(sp,ap)]
                        if n > 0 :
                            C[(s,a)] = (n/(n+1)) * C[(s,a)] + (1/(n+1)) * ((np.matmul(vt, mt))- m0 * sum(vt) + C[(sp,ap)])

                    else:
                        vt.append(v_remaining)
                        mt.append(Lp.iloc[0,10])
                        v_remaining = 0
                        ip = 0
                        sp = (t+1, ip)
                        temp = [C[(sp, j)] for j in A]
                        for k in A:
                            if C[(sp, k)] == min(temp):
                                ap = k
                        if n == 0:
                            C[(s,a)] = np.matmul(vt, mt) - vt[0] * m0 + C[(sp,ap)]
                        if n > 0:
                            C[(s,a)] =  (n/(n+1)) * C[(s,a)] + (1/(n+1)) * (np.matmul(vt, mt) - vt[0] * m0 + C[(sp,ap)])                            

                    

N :  15 time :  8
N :  15 time :  7
N :  15 time :  6
N :  15 time :  5
N :  15 time :  4
N :  15 time :  3
N :  15 time :  2
N :  15 time :  1
N :  15 time :  0
N :  16 time :  8
N :  16 time :  7
N :  16 time :  6
N :  16 time :  5
N :  16 time :  4
N :  16 time :  3
N :  16 time :  2
N :  16 time :  1
N :  16 time :  0


## Testing the model

In [403]:
Mean_prices = {}
Mean_prices["NFK_Price"] = []
Mean_prices["NFK_Price_volume"] = []
Mean_prices["IE_Price"] = []
Mean_prices["CP_Price"] = []
Mean_prices["SL_Price"] = []

## Start here
Run all bellow several times to get several results

In [1282]:
n += 1
for t in range(T,-1,-1):
        print("N : ", n, "time : ", t)
        Ls = obs.iloc[t*int(TT/T):(t+1)*int(TT/T),:]
        Lp = obp.iloc[t*int(TT/T):(t+1)*int(TT/T), :]
        O = df.iloc[t*int(TT/T):(t+1)*int(TT/T), :]

        for i in range(0, I+1):
            for a in A:
                s = (t, i)
                v_remaining = i*V/I
                if t == T: 
                    x = (v_remaining, 1)
                    C[(s,a)] = Lp.iloc[int(TT/T)-1, 10] * v_remaining - v_remaining * m0 
                else:
                    x = (v_remaining, a)
                    vt = []
                    mt = []

                    if Lp.iloc[0, 9] + a < Lp.iloc[0,10]:
                        for tt in range(t*int(TT/T), (t+1)*int(TT/T)):
                            if v_remaining > 0:
                                if ((O.loc[tt, "type"] == 4) & (O.loc[tt, "price"] <= Lp.iloc[0, 9] + a) & (O.loc[tt, "direction"] == 1)):
                                    vt.append(O.loc[tt, "size"])
                                    v_remaining -= O.loc[tt, "size"]
                                    mt.append(Lp.iloc[0, 9] + a)
                                if ((O.loc[tt, "type"] == 1) & (O.loc[tt, "price"] <= Lp.iloc[0, 9] + a) & (O.loc[tt, "direction"] == -1)):
                                    vt.append(O.loc[tt, "size"])
                                    v_remaining -= O.loc[tt, "size"]
                                    mt.append(O.loc[tt, "price"])
                        v_remaining = max (0, v_remaining)
                        ip = int(v_remaining/(V/I))

                        sp = (t+1, ip)
                        temp = [C[(sp, j)] for j in A]
                        for k in A:
                            if C[(sp, k)] == min(temp):
                                ap = k
                        if n == 0:
                            C[(s,a)] = (np.matmul(vt, mt))- m0 * sum(vt) + C[(sp,ap)]
                        if n > 0 :
                            C[(s,a)] = (n/(n+1)) * C[(s,a)] + (1/(n+1)) * ((np.matmul(vt, mt))- m0 * sum(vt) + C[(sp,ap)])

                    else:
                        vt.append(v_remaining)
                        mt.append(Lp.iloc[0,10])
                        v_remaining = 0
                        ip = 0
                        sp = (t+1, ip)
                        temp = [C[(sp, j)] for j in A]
                        for k in A:
                            if C[(sp, k)] == min(temp):
                                ap = k
                        if n == 0:
                            C[(s,a)] = np.matmul(vt, mt) - vt[0] * m0 + C[(sp,ap)]
                        if n > 0:
                            C[(s,a)] =  (n/(n+1)) * C[(s,a)] + (1/(n+1)) * (np.matmul(vt, mt) - vt[0] * m0 + C[(sp,ap)])                            


N :  47 time :  8
N :  47 time :  7
N :  47 time :  6
N :  47 time :  5
N :  47 time :  4
N :  47 time :  3
N :  47 time :  2
N :  47 time :  1
N :  47 time :  0


In [1283]:
import csv
with open('C.csv', 'w') as f:
    for key in C.keys():
        f.write("%s,%s\n"%(key,C[key]))
with open('n.txt', 'w') as f:
    f.write(str(n))

In [1284]:
obp = obp.iloc[4001:,:].reset_index(drop = True)
obs = obs.iloc[4001:,:].reset_index(drop = True)
df = df.iloc[4001:,:].reset_index(drop = True)

## NFK Model testing

In [1285]:
A = [i*300 for i in range (-10,11)]
t = 0

def test_scenario(C,I,T,A,t,obp,obs,df):
    t0 = t
    tp = t + 8
    i = 4
    Cp = {}
    m0 = int((obp.iloc[t0*int(TT/T), 9] +  obp.iloc[t0*int(TT/T), 10])/2)
    print("m0 : ", m0)
    while t <= tp and i > 0:
        print (t)
        s = (t,i)
        Ls = obs.iloc[t*int(TT/T):(t+1)*int(TT/T),:]
        Lp = obp.iloc[t*int(TT/T):(t+1)*int(TT/T), :]
        O = df.iloc[t*int(TT/T):(t+1)*int(TT/T), :]

        v_remaining = i*V/I
        a = 0
        temp = [C[(s, j)] for j in A]
        print(min(temp))
        for k in A:
            if C[(s, k)] == min(temp):
                a = k
                break

        vt = []
        mt = []
        if t == tp:
            i = 0
            Cp[(s,a)] = Lp.iloc[0, 10] * v_remaining - v_remaining * m0 
        if Lp.iloc[0,9] + a < Lp.iloc[0, 10]:
            
            for tt in range(t*int(TT/T), (t+1)*int(TT/T)):
                if v_remaining > 0 :
                    if ((O.loc[tt, "type"] == 4) & (O.loc[tt, "price"] <= Lp.iloc[0,9] + a) & (O.loc[tt, "direction"] == 1)):
                        vt.append(O.loc[tt, "size"])
                        v_remaining -= O.loc[tt, "size"]
                        mt.append(Lp.iloc[0, 9] + a)
                    if ((O.loc[tt, "type"] == 1) & (O.loc[tt, "price"] <= Lp.iloc[0,9] + a) & (O.loc[tt, "direction"] == -1)):
                        vt.append(O.loc[tt, "size"])
                        v_remaining -= O.loc[tt, "size"]
                        mt.append(O.loc[tt, "price"])
        
            v_remaining = max(0, v_remaining)
            i = int(v_remaining/(V/I))
            Cp[(s,a)] = np.matmul(vt, mt)- m0 * sum(vt)
            
        else:
            
            vt.append(v_remaining)
            mt.append(Lp.iloc[0,10])
            v_remaining = 0
            i = 0
            print(mt)
            Cp[(s,a)] = np.matmul(vt, mt) - sum(vt) * m0
            
        t += 1
        print(i)
        
    return(Cp)
        
    
    
    

## Testing Instant execution
La stratégie la plus simple consiste à soumettre un ordre marché pour le volume complet au début de l'épisode. Cela garantit que tout le volume est exécuté et réduit les risques liés aux variations défavorables de prix. Cependant, en vendant ou en achetant le volume total au début, l'investisseur obtiendra des prix pires que celui les ordres qui sont consommés dans le cote opposé du carnet d'ordres. IE agit comme une base de référence du pire des cas pour montrer ce qu'un ordre de marché des le début coûterait.

In [1286]:
import math
def Price_NFK(C,I,T,A,t,obp,obs,df):
    t0 = t
    tp = t + 8
    i = 4
    Cp = {}
    m0 = int((obp.iloc[t0*int(TT/T), 9] +  obp.iloc[t0*int(TT/T), 10])/2)
    print("m0 : ", m0)
    Prices = []
    Volumes = []
    while t <= tp and i > 0:
        print (t)
        s = (t,i)
        Ls = obs.iloc[t*int(TT/T):(t+1)*int(TT/T),:]
        Lp = obp.iloc[t*int(TT/T):(t+1)*int(TT/T), :]
        O = df.iloc[t*int(TT/T):(t+1)*int(TT/T), :]

        v_remaining = i*V/I
        a = 0
        temp = [C[(s, j)] for j in A]
        
        for k in A:
            if C[(s, k)] == min(temp):
                a = k
                break

        vt = []
        mt = []
        if t == tp:
            Volumes.append(i)
            Prices.append(Lp.iloc[0, 10])
            i = 0
            Cp[(s,a)] = Lp.iloc[0, 10] * v_remaining - v_remaining * m0
            break
            
        if Lp.iloc[0,9] + a < Lp.iloc[0, 10]:
            
            for tt in range(t*int(TT/T), (t+1)*int(TT/T)):
                if v_remaining > 0 :
                    if ((O.loc[tt, "type"] == 4) & (O.loc[tt, "price"] <= Lp.iloc[0,9] + a) & (O.loc[tt, "direction"] == 1)):
                        vt.append(O.loc[tt, "size"])
                        v_remaining -= O.loc[tt, "size"]
                        mt.append(Lp.iloc[0, 9] + a)
                    if ((O.loc[tt, "type"] == 1) & (O.loc[tt, "price"] <= Lp.iloc[0,9] + a) & (O.loc[tt, "direction"] == -1)):
                        vt.append(O.loc[tt, "size"])
                        v_remaining -= O.loc[tt, "size"]
                        mt.append(O.loc[tt, "price"])
        
            v_remaining = max(0, v_remaining)
            if mt != []:
                Prices.append(np.mean(mt))
            else:
                Prices.append(0)
            i0 = i
            i = int(v_remaining/(V/I))
            if vt != []:
                Volumes.append(i0 - i)
            else:
                Volumes.append(0)
            Cp[(s,a)] = np.matmul(vt, mt)- m0 * sum(vt)
            
        else:
            
            vt.append(v_remaining)
            mt.append(Lp.iloc[0,10])
            v_remaining = 0
            i = 0
            Prices.append(np.mean(mt))
            Volumes.append(math.ceil(sum(vt)/(V/I)))
            Cp[(s,a)] = np.matmul(vt, mt) - sum(vt) * m0
            
        t += 1
        print(i)
        
    return (Prices, Volumes)

In [1287]:
Prices_NF, Volumes_NF = Price_NFK(C,I,T,A,t,obp,obs,df)

m0 :  2213800
0
0


In [1289]:
NFK_Price = np.matmul(Prices_NF, Volumes_NF)/I

In [1290]:
print("The NFK mean Price is : ", NFK_Price)

The NFK mean Price is :  2214200.0


In [1291]:
Mean_prices["NFK_Price"].append(NFK_Price)

In [1292]:
IE_Price = obp.iloc[0,10]
print("The IE Price is : ", IE_Price)

The IE Price is :  2214200


In [1293]:
Mean_prices["IE_Price"].append(IE_Price)

## Submit and leave

In [1294]:
price = obp.iloc[0,9]
v_remaining = 10000
t = 0
mt = []
vt = []
for tt in range(t*int(TT/T), (t+T-1)*int(TT/T)):
    if v_remaining > 0 :
        if ((df.loc[tt, "type"] == 4) & (df.loc[tt, "price"] <= price) & (df.loc[tt, "direction"] == 1)):
            vt.append(df.loc[tt, "size"])
            v_remaining -= df.loc[tt, "size"]
            mt.append(price)
        if ((df.loc[tt, "type"] == 1) & (df.loc[tt, "price"] <= price) & (df.loc[tt, "direction"] == -1)):
            vt.append(df.loc[tt, "size"])
            v_remaining -= df.loc[tt, "size"]
            mt.append(df.loc[tt, "price"])
    
    v_remaining = max(0, v_remaining)

if v_remaining > 0:
    price = obp.iloc[4000,10]
    mt.append(price)
    vt.append(v_remaining)
    
    

In [1295]:
SL_Price = np.matmul(vt,mt)/sum(vt)
print("The submit and leave mean price is : ", SL_Price)

The submit and leave mean price is :  2219978.22


In [1296]:
Mean_prices["SL_Price"].append(SL_Price)

## Constant Policy

In [1297]:
v_remaining = 10000
mt = []
vt = []

for t in range(0,T):
    for tt in range(t*int(TT/T), (t+1)*int(TT/T)):
        price = obp.iloc[t*int(TT/T),9]
        if v_remaining > 0 :
            if ((df.loc[tt, "type"] == 4) & (df.loc[tt, "price"] <= price) & (df.loc[tt, "direction"] == 1)):
                vt.append(df.loc[tt, "size"])
                v_remaining -= df.loc[tt, "size"]
                mt.append(price)
            if ((df.loc[tt, "type"] == 1) & (df.loc[tt, "price"] <= price) & (df.loc[tt, "direction"] == -1)):
                vt.append(df.loc[tt, "size"])
                v_remaining -= df.loc[tt, "size"]
                mt.append(df.loc[tt, "price"])

        v_remaining = max(0, v_remaining)

if v_remaining > 0:
    price = obp.iloc[4000,10]
    mt.append(price)
    vt.append(v_remaining)

In [1298]:
CP_Price = np.matmul(vt,mt)/sum(vt)
print("The constant policy mean price is : ", CP_Price)

The constant policy mean price is :  2219914.22


In [1300]:
Mean_prices["CP_Price"].append(CP_Price)

## NFK Volume choosing

In [1301]:
A = [i*300 for i in range (-10,11)]
I = 4
V = 10000
T = 8
TT = 4000
t = 0

def Price_NFK_volume(C,I,T,A,t,obp,obs,df):
    t0 = t
    tp = t + T
    i = 4
    Cp = {}
    m0 = int((obp.iloc[t0*int(TT/T), 9] +  obp.iloc[t0*int(TT/T), 10])/2)
    print("m0 : ", m0)
    Prices = []
    Volumes = []
    
    while t <= tp and i > 0:
        
        print ("time : ",t)
        s = (t,i)
        Ls = obs.iloc[t*int(TT/T):(t+1)*int(TT/T),:]
        Lp = obp.iloc[t*int(TT/T):(t+1)*int(TT/T), :]
        O = df.iloc[t*int(TT/T):(t+1)*int(TT/T), :]
        v_remaining = i*V/I
        a = 0
        vb = i
        temp = [C[(s, j)] for j in A]
        
        temp = [C[((0,h), j)] for h in range(1,i) for j in A]

        for k in A:
            for v in range(1,i):
                if C[((t,v), k)] == min(temp):
                    a = k
                    vb = v
                    break
        print("volume : ",vb)
        
        vt = []
        mt = []
        
        if t == tp:
            
            Volumes.append(i)
            Prices.append(Lp.iloc[0, 10])
            i = 0
            Cp[(s,a)] = Lp.iloc[0, 10] * v_remaining - v_remaining * m0
            
        else:
            
            vb_remaining = vb*V/I

            if Lp.iloc[0,9] + a < Lp.iloc[0, 10]:

                for tt in range(t*int(TT/T), (t+1)*int(TT/T)):
                    if vb_remaining > 0 :
                        if ((O.loc[tt, "type"] == 4) & (O.loc[tt, "price"] <= Lp.iloc[0,9] + a) & (O.loc[tt, "direction"] == 1)):
                            vt.append(O.loc[tt, "size"])
                            vb_remaining -= O.loc[tt, "size"]
                            mt.append(Lp.iloc[0, 9] + a)
                        if ((O.loc[tt, "type"] == 1) & (O.loc[tt, "price"] <= Lp.iloc[0,9] + a) & (O.loc[tt, "direction"] == -1)):
                            vt.append(O.loc[tt, "size"])
                            vb_remaining -= O.loc[tt, "size"]
                            mt.append(O.loc[tt, "price"])

                vb_remaining = max(0, vb_remaining)
                if mt != []:
                    Prices.append(np.mean(mt))
                else:
                    Prices.append(0)
                i0 = i
                i = i - vb + int(vb_remaining/(V/I))
                print("remaining volume:", i)
                if vt != []:
                    Volumes.append(i0 - i)
                else:
                    Volumes.append(0)
                Cp[(s,a)] = np.matmul(vt, mt)- m0 * sum(vt)

            else:

                vt.append(vb_remaining)
                mt.append(Lp.iloc[0,10])
                v_remaining = 0
                i = i - vb 
                Prices.append(np.mean(mt))
                Volumes.append(math.ceil(sum(vt)/(V/I)))
                Cp[(s,a)] = np.matmul(vt, mt) - sum(vt) * m0
            
        t += 1
        print(i)
        
    return (Prices, Volumes)

In [1302]:
Prices_NF_volume, Volumes_NF_volume = Price_NFK_volume(C,I,T,A,t,obp,obs,df)

m0 :  2213800
time :  0
volume :  3
1
time :  1
volume :  1
remaining volume: 0
0


In [1303]:
Prices_NF_volume

[2214200.0, 2213600.0]

In [1304]:
Volumes_NF_volume

[3, 1]

In [1305]:
NFK_Price_volume = np.matmul(Prices_NF_volume, Volumes_NF_volume)/I

In [1306]:
print("The NFK with volume choice mean price is : ", NFK_Price_volume)

The NFK with volume choice mean price is :  2214050.0


In [1307]:
Mean_prices["NFK_Price_volume"].append(NFK_Price_volume)

In [1308]:
pd.DataFrame(Mean_prices)/1000

,NFK_Price,NFK_Price_volume,IE_Price,CP_Price,SL_Price
0,2246.111700,2245.675000,2246.9,2244.344130,2244.267100
1,2242.900000,2245.442045,2244.0,2246.161130,2246.269660
2,2246.300000,2246.350000,2246.3,2245.890900,2245.698500
3,2244.378571,2240.601705,2245.7,2241.875658,2243.171687
4,2241.100000,2240.601705,2241.1,2238.933410,2239.062593
5,2237.900000,2237.175000,2237.9,2236.823570,2236.371024
6,2236.780000,2236.128125,2237.6,2235.716640,2235.302614
7,2235.900000,2235.683333,2235.9,2237.454580,2237.700000
8,2237.700000,2237.900000,2237.7,2237.640260,2237.667450
9,2237.700000,2237.175000,2237.7,2236.748180,2235.625886


In [1309]:
(pd.DataFrame(Mean_prices)/1000).mean()

NFK_Price           2228.573052
NFK_Price_volume    2228.213109
IE_Price            2228.722581
CP_Price            2228.298390
SL_Price            2228.538864
dtype: float64

In [1310]:
(pd.DataFrame(Mean_prices)/1000).to_csv("Mean_Prices.csv")